In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=e90f11039b41cd50927a090aee216f3d686ddb30ed0e591031e6f2a9fa8455c3
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [2]:
import wandb
from wandb.keras import WandbCallback

# If you're running this in Google Colab or Jupyter Notebook, you can use this line to log in to WandB
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [4]:
import pandas as pd
# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True, connected=True)

import os

import json
from tqdm import tqdm
import numpy as np
import itertools
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_disable_xla_devices'
import tensorflow as tf

# #pytorch model
# import torch
# import torch.nn.functional as F
# import torch.nn as nn

import seaborn as sns

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from matplotlib import animation
from pathlib import Path
import IPython
from IPython import display
from IPython.core.display import display, HTML, Javascript
from IPython.display import Markdown as md

# import mediapipe as mp
# from mediapipe.framework.formats import landmark_pb2

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_disable_xla_devices'

Num GPUs Available:  1


In [ ]:
# #only run this if data_chunks doest exist
# import numpy as np
# import os
# from tqdm import tqdm

# def save_chunks(data, labels, chunk_size, output_dir):
#     num_samples = len(data)
#     num_chunks = (num_samples + chunk_size - 1) // chunk_size
    
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     for i in tqdm(range(num_chunks), desc="Saving Chunks"):
#         start = i * chunk_size
#         end = min((i + 1) * chunk_size, num_samples)
        
#         chunk_data = data[start:end]
#         chunk_labels = labels[start:end]
        
#         np.save(os.path.join(output_dir, f'feature_data_{i}.npy'), chunk_data)
#         np.save(os.path.join(output_dir, f'feature_labels_{i}.npy'), chunk_labels)

# # Update the file paths to point to your Google Drive
# drive_path = '/content/drive/My Drive/W251_Project/'

# x_data = np.load(os.path.join(drive_path, 'feature_data.npy'))
# y_data = np.load(os.path.join(drive_path, 'feature_labels.npy'))
# chunk_size = 1000  # Adjust this value based on your memory constraints
# output_dir = os.path.join(drive_path, 'data_chunks')

# save_chunks(x_data, y_data, chunk_size, output_dir)


In [6]:
drive_path = '/content/drive/My Drive/W251_Project/'
output_dir = os.path.join(drive_path, 'data_chunks')

In [7]:
import tensorflow as tf
from glob import glob
from sklearn.model_selection import train_test_split

def data_generator(x_file_paths, y_file_paths):
    for x_file_path, y_file_path in zip(x_file_paths, y_file_paths):
        x_data = np.load(x_file_path)
        y_data = np.load(y_file_path)

        for x, y in zip(x_data, y_data):
            yield np.transpose(x, (2, 0, 1)), y

def create_dataset(x_file_paths, y_file_paths, batch_size):
    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(x_file_paths, y_file_paths),
        output_signature=(
            tf.TensorSpec(shape=(3, 37, 192), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset


x_file_paths = sorted(glob(os.path.join(output_dir, 'feature_data_*.npy')))
y_file_paths = sorted(glob(os.path.join(output_dir, 'feature_labels_*.npy')))
file_paths = list(zip(x_file_paths, y_file_paths))

train_file_paths, test_file_paths = train_test_split(file_paths, test_size=0.2, random_state=42)
train_x_file_paths, train_y_file_paths = zip(*train_file_paths)
test_x_file_paths, test_y_file_paths = zip(*test_file_paths)

batch_size = 32
train_dataset = create_dataset(train_x_file_paths, train_y_file_paths, batch_size)
test_dataset = create_dataset(test_x_file_paths, test_y_file_paths, batch_size)


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Flatten

def transformer_encoder(input_shape, num_heads, ff_dim, dropout_rate):
    inputs = Input(shape=tuple(input_shape))
    attn_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=input_shape[-1])(inputs, inputs)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)
    ff_output = Dense(ff_dim, activation='relu')(out1)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(input_shape[-1])(ff_output)  # Add this line to match the dimensions
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ff_output)
    model = tf.keras.models.Model(inputs=inputs, outputs=out2)
    return model


def create_spbt(input_shape, num_classes, num_heads, ff_dim, dropout_rate, num_encoders):
    inputs = Input(shape=input_shape)
    x = inputs
    encoder_input_shape = input_shape
    for _ in range(num_encoders):
        x = transformer_encoder(encoder_input_shape, num_heads, ff_dim, dropout_rate)(x)
        encoder_input_shape = x.shape[1:]  # Update the input shape for the next encoder
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    return model



In [9]:


from tensorflow.keras.optimizers import Adam
from glob import glob


def get_all_x_y_file_paths(x_file_pattern, y_file_pattern):
    x_file_paths = sorted(glob(x_file_pattern))
    y_file_paths = sorted(glob(y_file_pattern))
    
    if len(x_file_paths) != len(y_file_paths):
        raise ValueError("The number of X and y files does not match.")
    
    return list(zip(x_file_paths, y_file_paths))


# Function to extract unique class count from the dataset
def get_unique_class_count(dataset):
    unique_classes = set()
    for _, label in dataset.unbatch().as_numpy_iterator():
        unique_classes.add(label)
    return len(unique_classes)

# Get all X and y file paths
all_file_paths = get_all_x_y_file_paths(os.path.join(drive_path, 'data_chunks/feature_data_*.npy'), os.path.join(drive_path, 'data_chunks/feature_labels_*.npy'))


# Split the file paths into training and testing sets
train_file_paths, test_file_paths = train_test_split(all_file_paths, test_size=0.2, random_state=42)

# Separate X and y file paths for train and test datasets
train_x_file_paths, train_y_file_paths = zip(*train_file_paths)
test_x_file_paths, test_y_file_paths = zip(*test_file_paths)

# Define batch size
batch_size = 64

# Create train and test datasets
train_dataset = create_dataset(train_x_file_paths, train_y_file_paths, batch_size)
test_dataset = create_dataset(test_x_file_paths, test_y_file_paths, batch_size)

# Define model parameters
input_dim = 192 * 3
# num_classes = get_unique_class_count(train_dataset)
num_classes = 250
num_heads = 9
ff_dim = 2048
dropout_rate = 0.3
num_encoders = 6
epochs=20


In [10]:
# Set up WandB configuration
config = dict(
    batch_size=batch_size,
    num_classes=num_classes,
    num_heads=num_heads,
    ff_dim=ff_dim,
    dropout_rate=dropout_rate,
    num_encoders=num_encoders,
)

# Initialize WandB run
wandb.init(project='W251_finalproject', config=config)


wandb: Currently logged in as: reynard011 (w251-asl-fp). Use `wandb login --relogin` to force relogin


In [ ]:
# class WarmUpCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
#     def __init__(self, initial_learning_rate, warmup_steps, total_steps, alpha=0.0):
#         super(WarmUpCosineDecay, self).__init__()
        
#         self.initial_learning_rate = initial_learning_rate
#         self.warmup_steps = warmup_steps
#         self.total_steps = total_steps
#         self.alpha = alpha
    
#     def __call__(self, step):
#         step_float = tf.cast(step, tf.float32)
#         warmup_steps_float = tf.cast(self.warmup_steps, tf.float32)

#         def warmup_phase():
#             return step_float * self.initial_learning_rate / warmup_steps_float
        
#         def cosine_decay_phase():
#             decay_steps = tf.math.maximum(step_float - warmup_steps_float, 1)
#             decay_steps = tf.math.minimum(decay_steps, self.total_steps)
#             cosine_decay = 0.5 * (1 + tf.math.cos(tf.constant(np.pi) * decay_steps / self.total_steps))
#             decayed = (1 - self.alpha) * cosine_decay + self.alpha
#             return self.initial_learning_rate * decayed
        
#         return tf.cond(step_float < warmup_steps_float, warmup_phase, cosine_decay_phase)
    
#     def get_config(self):
#         return {
#             "initial_learning_rate": self.initial_learning_rate,
#             "warmup_steps": self.warmup_steps,
#             "total_steps": self.total_steps,
#             "alpha": self.alpha,
#         }

# # Example usage:
# initial_learning_rate = 0.001
# warmup_steps = 1000
# total_steps = 10000
# alpha = 0.0001

# lr_schedule = WarmUpCosineDecay(initial_learning_rate, warmup_steps, total_steps, alpha)
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile and train your model
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(train_dataset, epochs=epochs, validation_data=test_dataset, verbose=1)


In [11]:
import tensorflow as tf

initial_learning_rate = 0.0001
decay_steps = 32150
alpha = 0.0001

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate,
    decay_steps,
    alpha=alpha
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
model = create_spbt(input_shape=(3, 37, 192), num_classes=num_classes, num_heads=num_heads, ff_dim=ff_dim, dropout_rate=dropout_rate, num_encoders=num_encoders)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 37, 192)]      0         
                                                                 
 model (Functional)          (None, 3, 37, 192)        2121920   
                                                                 
 model_1 (Functional)        (None, 3, 37, 192)        2121920   
                                                                 
 model_2 (Functional)        (None, 3, 37, 192)        2121920   
                                                                 
 model_3 (Functional)        (None, 3, 37, 192)        2121920   
                                                                 
 model_4 (Functional)        (None, 3, 37, 192)        2121920   
                                                                 
 model_5 (Functional)        (None, 3, 37, 192)        2121

In [13]:
# Train the model
history=model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=test_dataset,verbose=1,
    callbacks=[WandbCallback()]
)


# history = model.fit(train_dataset, epochs=epochs, validation_data=test_dataset, verbose=1)


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20
   1179/Unknown - 152s 76ms/step - loss: 4.2371 - accuracy: 0.1457

wandb: Adding directory to artifact (/content/wandb/run-20230414_023526-vr5x8c0d/files/model-best)... Done. 0.7s


1180/1180 [==============================] - 197s 114ms/step - loss: 4.2367 - accuracy: 0.1458 - val_loss: 2.7822 - val_accuracy: 0.3671
Epoch 2/20
1179/1180 [============================>.] - ETA: 0s - loss: 2.2962 - accuracy: 0.4479

wandb: Adding directory to artifact (/content/wandb/run-20230414_023526-vr5x8c0d/files/model-best)... Done. 0.7s


...

1180/1180 [==============================] - 95s 77ms/step - loss: 1.3642 - accuracy: 0.6530 - val_loss: 1.5620 - val_accuracy: 0.6210
Epoch 5/20
1179/1180 [============================>.] - ETA: 0s - loss: 1.1088 - accuracy: 0.7125

wandb: Adding directory to artifact (/content/wandb/run-20230414_023526-vr5x8c0d/files/model-best)... Done. 0.7s


1180/1180 [==============================] - 95s 77ms/step - loss: 1.1090 - accuracy: 0.7125 - val_loss: 1.4757 - val_accuracy: 0.6470
Epoch 6/20
1179/1180 [============================>.] - ETA: 0s - loss: 0.9184 - accuracy: 0.7575

wandb: Adding directory to artifact (/content/wandb/run-20230414_023526-vr5x8c0d/files/model-best)... Done. 0.7s


1180/1180 [==============================] - 96s 78ms/step - loss: 0.9184 - accuracy: 0.7575 - val_loss: 1.4402 - val_accuracy: 0.6657
Epoch 7/20
1180/1180 [==============================] - 81s 65ms/step - loss: 0.7526 - accuracy: 0.7963 - val_loss: 1.5181 - val_accuracy: 0.6664
Epoch 8/20
1180/1180 [==============================] - 80s 65ms/step - loss: 0.6149 - accuracy: 0.8326 - val_loss: 1.4729 - val_accuracy: 0.6807
Epoch 9/20
1180/1180 [==============================] - 81s 65ms/step - loss: 0.4952 - accuracy: 0.8625 - val_loss: 1.4900 - val_accuracy: 0.6925
Epoch 10/20
1180/1180 [==============================] - 81s 65ms/step - loss: 0.3963 - accuracy: 0.8881 - val_loss: 1.5059 - val_accuracy: 0.6965
Epoch 11/20
1180/1180 [==============================] - 81s 65ms/step - loss: 0.3123 - accuracy: 0.9110 - val_loss: 1.5118 - val_accuracy: 0.7021
Epoch 12/20
1180/1180 [==============================] - 81s 65ms/step - loss: 0.2417 - accuracy: 0.9302 - val_loss: 1.5828 - val_acc

In [ ]:
print(tf.__version__)


2.12.0
